In [4]:
import pandas as pd
df1 = pd.read_csv(r"C:\Users\ymf\Desktop\pic16bGit\rye_data_project_spring2021\Reddit_dataset\Adulting2016-2021.csv")
df1.head(5)

,Unnamed: 0,author,created_utc,domain,full_link,num_comments,over_18,score,selftext,subreddit,title,num_crossposts
0,0,paradisenoir,1469810090,self.Adulting,https://www.reddit.com/r/Adulting/comments/4v7...,1,False,1,I seriously didn't know where to put this ques...,Adulting,Need help with Updating driver's License,NaN
1,1,emptysometimes,1469550546,youtu.be,https://www.reddit.com/r/Adulting/comments/4up...,0,False,1,NaN,Adulting,Adulting 101,NaN
2,2,PortlandPerson94,1468814980,self.Adulting,https://www.reddit.com/r/Adulting/comments/4td...,1,False,4,"How to do laundry, organize your filing cabine...",Adulting,"in the spirit of the discription of this sub, ...",NaN
3,3,thisisthetyty,1467816249,youtube.com,https://www.reddit.com/r/Adulting/comments/4ri...,0,False,1,NaN,Adulting,New youtube video about symptoms of adulting. ...,NaN
4,4,writer318,1467729176,theodysseyonline.com,https://www.reddit.com/r/Adulting/comments/4rc...,0,False,1,NaN,Adulting,"Why Must We Have To ""Adult""?",NaN


In [7]:
X=pd.DataFrame({'text':df1['selftext']})
X = X.dropna()
X

,text
0,I seriously didn't know where to put this ques...
2,"How to do laundry, organize your filing cabine..."
5,So the short version of the story my fiance an...
6,I am tired of myself and other adults not be o...
8,Finally mastering public transportation. I fee...
...,...
9394,Like which info do I give them? My policy numb...
9395,I seriously didn't know where to put this ques...
9397,"How to do laundry, organize your filing cabine..."
9400,So the short version of the story my fiance an...


In [14]:
import nltk
from nltk.corpus import stopwords
#nltk.download("stopwords") #uncomment it when run it for the first time
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text
X['text']=X['text'].apply(lambda x: remove_stopwords(x))


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer(max_df=0.5,min_df=0.01, stop_words='english')
counts=vec.fit_transform(X['text'])
counts=counts.toarray()
count_df1=pd.DataFrame(counts,columns=vec.get_feature_names())
count_df1=count_df1.drop(['amp','don'],axis=1)
count_df1

,000,10,100,12,15,16,17,18,19,20,...,wrong,www,x200b,yeah,year,years,yes,yesterday,young,younger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,0
7118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
from sklearn.decomposition import NMF

model1=NMF(n_components=4,init="random",random_state=0)
model1.fit(count_df1)
#model1.components_

NMF(init='random', n_components=4, random_state=0)

In [17]:
import numpy as np
def top_words(X, model, component, num_words):
    """
    Extract the top words from the specified component 
    for a topic model trained on data. 
    X: a term-document matrix, assumed to be a pd.DataFrame
    model: a sklearn model with a components_ attribute, e.g. NMF
    component: the desired component, specified as an integer. 
        Must be less than than the total number of components in model
    num_words: the number of words to return.
    """
    orders = np.argsort(model.components_, axis = 1)
    important_words = np.array(X.columns)[orders]
    return important_words[component][-num_words:]

In [21]:
topic1=pd.DataFrame({'Topic 0':top_words(count_df1, model1, 0, 10),
                   'Topic 1':top_words(count_df1, model1, 1, 10),
                   'Topic 2':top_words(count_df1, model1, 2, 10),
                   'Topic 3':top_words(count_df1, model1, 3, 10)})
topic1

,Topic 0,Topic 1,Topic 2,Topic 3
0,good,mom,day,apartment
1,friends,need,got,house
2,make,parents,school,rent
3,really,things,week,home
4,people,people,hours,insurance
5,time,life,working,need
6,life,really,ve,money
7,things,ve,work,pay
8,feel,want,time,credit
9,like,know,job,car
